In [ ]:
%%writefile chatgpt.py
import requests
import json
import time
import pandas as pd
from IPython.display import Markdown  


class APIPool:
    def __init__(self,tokens):
        self.tokens=tokens
        self.last_called={token:0 for token in tokens}  #上次调用时间
    def getapi(self):
        now = int(time.time())
        for token in self.tokens:
            if self.last_called[token]+21 <=now:    
                self.last_called[token]=now
                return token
        time.sleep(self.last_called[self.tokens[0]]+21-now) #等待
        return self.getapi() #重新调用
    def reset(self):
        self.last_called={token:0 for token in self.tokens}#重置


def get_header():# 获取请求头
    header={
  'Authorization': f'Bearer {A.getapi()}',
  'Content-Type': 'application/json'
    }
    return header

#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "system", "content": f"You are ChatGPT, a large language model trained by OpenAI. \
            Follow the user's instructions carefully. Respond using markdown."},
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    data=None
    try:
        response = requests.post(url, headers=get_header(), json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

url = "https://api.lixining.com/v1/chat/completions"

api_keys = ["sk-1","sk-2"]
A=APIPool(api_keys) 

def ask(prompt):
    
    result=(get_completion(prompt))
    #print(result)
    return result

from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)
 
@magics_class

class ChatGPTMagics(Magics):
 
    @line_magic
    def chat(self, line):
        return ask(line)
 
    @cell_magic
    def gpt(self, line, cell):
        result = ask(cell)
        #print(result)
 
    @line_cell_magic
    def chatgpt(self, line, cell=None):
        "Magic that works both as %chatgpt and as %%chatgpt"
        if cell is None:
            return ask(line)
        else:
            return Markdown(ask(cell))
 

# In order to actually use these magics, you must register them with a
# running IPython.
def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    ipython.register_magics(ChatGPTMagics)

In [ ]:
%load_ext chatgpt

In [ ]:
%%chatgpt
哈哈aaa